In [1]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [1]:
import keras
import tensorflow as tf

print("Keras Version:", keras.__version__)
print("Tensorflow Version:", tf.__version__)

Keras Version: 3.0.5
Tensorflow Version: 2.16.1


In [2]:
# Import necessary modules and functions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from src.neural_nets import create_neural_1, create_neural_3, create_neural_6, parameter_tuning_nn, runs_nn
from src.knn_regression import parameter_tuning_knn, runs_knn
from data.data_generator import get_data, preprocess

Regression function and data for hyperparameter tuning of the networks

In [3]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

Neural Network with one hidden Layer

In [4]:
# Hyperparameter tuning of network with 1 hidden layer

units = [5,10,25,50,75]
best_hp = parameter_tuning_nn(
    create_neural_1,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs = 1000,
)

print(best_hp)

Unit: 5, MSE: 0.2168329656124115, MAE: 0.3897862732410431
Unit: 10, MSE: 0.10799791663885117, MAE: 0.2666034400463104
Unit: 25, MSE: 0.0902738943696022, MAE: 0.24648132920265198
Unit: 50, MSE: 0.13269713521003723, MAE: 0.2800270617008209
Unit: 75, MSE: 0.1240115612745285, MAE: 0.2723569869995117
{'best_hp': 25, 'mse': 0.1240115612745285}


In [5]:
unit = best_hp['best_hp']
mses_neural_1, maes_neural_1 = runs_nn(
    create_neural_1,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [6]:
results_neural_1 = pd.DataFrame(
    {
        "MSE": mses_neural_1,
        "MAE": maes_neural_1,
    }
)

median_neural_1 = results_neural_1.median()
iqr_nerual_1 = results_neural_1.quantile(0.75) - results_neural_1.quantile(0.25)

results_neural_1.to_csv("results/neural_1.csv", index=False)

Neural Network with three hidden layers

In [7]:
# Hyperparameter tuning of network with 3 hidden layer

units = [3, 6, 9, 12, 15]
best_hp = parameter_tuning_nn(
    create_neural_3,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs=1000,
)

print(best_hp)

Unit: 3, MSE: 0.0658610537648201, MAE: 0.21022877097129822
Unit: 6, MSE: 0.12469211965799332, MAE: 0.2777542471885681
Unit: 9, MSE: 0.07162357121706009, MAE: 0.20529866218566895
Unit: 12, MSE: 0.12747475504875183, MAE: 0.3189464211463928
Unit: 15, MSE: 0.1181844025850296, MAE: 0.26013341546058655
{'best_hp': 3, 'mse': 0.1181844025850296}


In [8]:
unit = best_hp['best_hp']
mses_neural_3, maes_neural_3 = runs_nn(
    create_neural_3,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [9]:
results_neural_3 = pd.DataFrame(
    {
        "MSE": mses_neural_3,
        "MAE": maes_neural_3,
    }
)

median_neural_3 = results_neural_3.median()
iqr_nerual_3 = results_neural_3.quantile(0.75) - results_neural_3.quantile(0.25)

results_neural_3.to_csv("results/neural_3.csv", index=False)

Neural Network with six hidden layers

In [10]:
# Hyperparameter tuning of network with 6 hidden layer

units = [2, 4, 6, 8, 10]
best_hp = parameter_tuning_nn(
    create_neural_6,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units,
    epochs=1000,
)

print(best_hp)

Unit: 2, MSE: 5.333621978759766, MAE: 1.8797540664672852
Unit: 4, MSE: 0.06482017785310745, MAE: 0.2042514979839325
Unit: 6, MSE: 0.08910302072763443, MAE: 0.24334374070167542
Unit: 8, MSE: 0.20454709231853485, MAE: 0.36493217945098877
Unit: 10, MSE: 0.11168136447668076, MAE: 0.2055419385433197
{'best_hp': 4, 'mse': 0.11168136447668076}


In [11]:
unit = best_hp['best_hp']
mses_neural_6, maes_neural_6 = runs_nn(
    create_neural_6,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=1000,
    batch_size=20,
    )

In [12]:
results_neural_6 = pd.DataFrame(
    {
        "MSE": mses_neural_6,
        "MAE": maes_neural_6,
    }
)

median_neural_6 = results_neural_6.median()
iqr_nerual_6 = results_neural_6.quantile(0.75) - results_neural_6.quantile(0.25)

results_neural_6.to_csv("results/neural_6.csv", index=False)

kNN Regression

In [4]:
# Preprocess data
train_data = (x_train, y_train)
test_data = (x_val, y_val)

# Tune number of neighbors
u = [1, 2, 3]
v = [i for i in range(4, x_train.shape[0]+1, 4)]
units = u + v
best_params = parameter_tuning_knn(
    units=units,
    train_data=train_data,
    test_data=test_data,
)

# Calculate regression for the best number of neighbors
best_units = best_params["best_config"]
knn_mses, knn_maes = runs_knn(
    unit=best_units,
    input_dim=input_dim,
    regression_func=regression_func,
)

Unit: 1, MSE: 1.0684580712367673, MAE: 0.7683278653402889
Unit: 2, MSE: 0.7584612835123383, MAE: 0.6802976742432603
Unit: 3, MSE: 1.0370811383495164, MAE: 0.8423768007038621
Unit: 4, MSE: 0.8248808066537725, MAE: 0.7452062394329765
Unit: 8, MSE: 0.7809706135667434, MAE: 0.7074845799948501
Unit: 12, MSE: 0.7183030296695786, MAE: 0.7022738817688879
Unit: 16, MSE: 0.8809886997044984, MAE: 0.7834409141864931
Unit: 20, MSE: 0.92385741014287, MAE: 0.8175227746934179
Unit: 24, MSE: 1.0236330435262897, MAE: 0.8731860991465485
Unit: 28, MSE: 1.125868602738458, MAE: 0.8897779620013914
Unit: 32, MSE: 1.1816884645202443, MAE: 0.909025525805038
Unit: 36, MSE: 1.2955915338788455, MAE: 0.9536674153164346
Unit: 40, MSE: 1.3824764972133898, MAE: 0.9840291982365361
Unit: 44, MSE: 1.4558678278589343, MAE: 1.0116164567303703
Unit: 48, MSE: 1.4933857029070194, MAE: 1.014201963944672
Unit: 52, MSE: 1.5679071070363704, MAE: 1.0330762377593372
Unit: 56, MSE: 1.6679374794248556, MAE: 1.0618032648130822
Unit: 6

In [6]:
results_knn = pd.DataFrame(
    {
        "MSE": knn_mses,
        "MAE": knn_maes,
    }
)

median_knn = results_knn.median()
iqr_knn = results_knn.quantile(0.75) - results_knn.quantile(0.25)

results_knn.to_csv("results/knn.csv", index=False)